# Análise Exploratória: Consumo de Álcool e Desempenho de Estudantes

Grupo:
- João Vitor Azevedo
- Luísa Tavares dos Santos


## Introdução

O sucesso (ou fracasso) de um estudante no ensino médio raramente depende apenas de quanto tempo ele passa estudando. Fatores sociais, familiares e hábitos pessoais criam um ecossistema complexo que influencia diretamente o seu desempenho. Este projeto utiliza o famoso dataset Student Alcohol Consumption, que detalha a vida de estudantes de ensino secundário em Portugal.

O conjunto de dados nos fornece informações que vão desde o trabalho dos pais e a qualidade do relacionamento familiar até hábitos como o consumo de álcool e a frequência de saídas com amigos, cruzando tudo isso com as notas obtidas nas disciplinas de Matemática e Português.

Nosso principal objetivo com esta Análise Exploratória de Dados (AED) é investigar essas conexões. Queremos usar os dados para responder a perguntas como: "Qual o verdadeiro impacto do consumo de álcool nas notas?" e "Como o estilo de vida social de um aluno se relaciona com sua performance acadêmica?". Ao final, buscamos apresentar um panorama claro dos fatores que mais contribuem para o sucesso e para o risco na vida escolar desses jovens.

Para isso, vamos passar pelas etapas de preparação dos dados, análises univariada e bivariada, e investigar algumas hipóteses para descobrir padrões interessantes.

## 1. Carregando e Conhecendo os Dados

Para começar, vamos configurar nosso ambiente de trabalho, carregar os dados e fazer uma primeira inspeção para entender a estrutura geral do dataset.

### 1.1. Configurando o Ambiente

In [ ]:
# Importação das bibliotecas padrão
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

# Configurações de visualização para os gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

### 1.2. Descrição e Carregamento dos Dados

- **Nome do Dataset:** Student Alcohol Consumption
- **Fonte/Link:** https://www.kaggle.com/datasets/uciml/student-alcohol-consumption/data
- **Descrição:** Este dataset contém informações sobre estudantes de duas escolas secundárias em Portugal, coletadas através de relatórios escolares e questionários. Inclui notas, características demográficas, sociais e relacionadas ao consumo de álcool.

O dataset vem dividido em dois arquivos: **student-mat.csv** (com dados de estudantes de Matemática) e **student-por.csv** (de Português).

A existência de dois arquivos separados para Matemática e Português contém um desafio: muitos alunos aparecem em ambos. Analisá-los separadamente seria um erro, pois estaríamos contando o mesmo aluno duas vezes e teríamos uma visão incompleta de sua vida acadêmica. Portanto, a unificação dos datasets não é apenas uma conveniência, mas uma etapa essencial para garantir que nossa análise seja precisa

In [ ]:
# Carregamento dos dois datasets
path = kagglehub.dataset_download("uciml/student-alcohol-consumption")
file_path_math = path + '/student-mat.csv'
file_path_port = path + '/student-por.csv'
try:
    df_mat = pd.read_csv(file_path_math)
    df_por = pd.read_csv(file_path_port)
    print('Datasets de Matemática e Português carregados com sucesso!')
except FileNotFoundError:
    print("Erro: Verifique se os arquivos 'student-mat.csv' e 'student-por.csv' estão no diretório.")

## Perguntas Principais
- Qual a relação entre o consumo de álcool e o desempenho acadêmico?
- Como o estilo de vida social e os hábitos (sair, faltar) se conectam com o desempenho?
- Quais outros fatores demonstram forte influência sobre o sucesso acadêmico?

## 2. Preparação e Limpeza dos Dados

Agora que os dados estão carregados, a próxima etapa é a preparação. Vamos primeiro unificar os dois datasets

### 2.1. Unificação dos Datasets

In [ ]:
# Primeiro, utilizaremos o comando info em cada dataset para verificar sobre ele antes de unifica-los
print(f'informações sobre o dataset de matemática')
df_mat.info()

In [ ]:
print(f'informações sobre o dataset de português')
df_por.info()

### Explicação sobre o que significam os atributos

Para entender nossas análises, é importante conhecer as variáveis que temos em mãos. Elas se dividem em alguns grupos principais:

Dados Demográficos:

- school: Escola do estudante (GP - Gabriel Pereira ou MS - Mousinho da Silveira).
- sex: Gênero ('F' - Feminino, 'M' - Masculino).
- age: Idade (de 15 a 22 anos).
- address: Tipo de endereço ('U' - Urbano, 'R' - Rural).
- famsize: Tamanho da família ('LE3' - Menor ou igual a 3, 'GT3' - Maior que 3).
- Pstatus: Status de coabitação dos pais ('T' - Moram juntos, 'A' - Separados).

Dados Familiares e de Suporte:

- Medu, Fedu: Nível de educação da mãe e do pai (0 a 4, onde 4 é ensino superior).
- Mjob, Fjob: Trabalho da mãe e do pai.
- guardian: Guardião legal do estudante.
- famsup: Suporte educacional da família (sim/não).
- schoolsup: Suporte educacional extra (reforço) (sim/não).
- paid: Aulas particulares pagas na disciplina (sim/não).
- famrel: Qualidade do relacionamento familiar (1 - muito ruim a 5 - excelente).

Hábitos e Estilo de Vida:

- studytime: Tempo de estudo semanal (1 a 4, representando faixas de horas).
- traveltime: Tempo de viagem de casa para a escola (1 a 4).
- goout: Frequência de saídas com amigos (1 - muito baixa a 5 - muito alta).
- Dalc, Walc: Consumo de álcool durante a semana e no fim de semana (1 - muito baixo a 5 - muito alto).
- health: Estado de saúde atual (1 - muito ruim a 5 - muito bom).
- activities: Participa de atividades extracurriculares (sim/não).
- internet: Possui acesso à internet em casa (sim/não).
- romantic: Está em um relacionamento amoroso (sim/não).

Dados Acadêmicos:

- failures: Número de reprovações em classes anteriores.
- absences: Número de faltas escolares.
- G1, G2: Notas do primeiro e segundo período (0 a 20).
- G3: Nota final (0 a 20) - nossa principal variável de resultado.

### Unificação

A gente percebeu que uma simples remoção de duplicatas (drop_duplicates) seria arbitrária e nos faria perder informações valiosas (por exemplo, manteríamos as notas de matemática e jogaríamos fora as de português para um mesmo aluno).

Então fizemos por partes:

- Identificação: Primeiro, identificamos todos os registros que pertenciam a alunos duplicados (aqueles que aparecem nos dois arquivos).
- Separação: Dividimos nosso conjunto de dados em dois grupos: um com os alunos "únicos" (que só cursaram uma matéria) e outro com os alunos "duplicados".
- Agregação: Para o grupo de duplicados, nós agregamos suas informações. Para as colunas numéricas que variam entre as disciplinas, como as notas (G1, G2, G3) e as faltas (absences), calculamos a média entre as duas matérias. Para informações que são fixas (como idade, sexo, trabalho dos pais), simplesmente mantivemos o primeiro registro.
- União Final: Por fim, juntamos o grupo de alunos únicos com o nosso novo grupo de alunos agregados. O resultado é um dataframe final e coeso, sem duplicidade e que aproveita o máximo de informação de cada estudante.



In [ ]:
# Unindo os dois dataframes, como seguem a mesma estrutura de atributos, não é necessária manipulação antes de uní-los e remover as duplicatas
df = pd.concat([df_mat, df_por], ignore_index=True)

# As colunas que identificam um aluno de forma única (definido na coleta dos dados)
colunas_identificadoras_aluno = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]

# Identificando alunos duplicados (estão tanto nas aulas de português como nas de matemática) com uma coluna booleana
indices_duplicados = df.duplicated(subset=colunas_identificadoras_aluno, keep=False)

# Vamos separar o dataframe em dois, o que tem alunos nas duas turmas e o que só tem alunos únicos(apenas em uma turma), para identificar dados que precisam de tratamento
df_alunos_duplicados = df[indices_duplicados]
df_alunos_unicos = df[~indices_duplicados]

print(f'Número de registros no dataframe de Matemática: {len(df_mat)}')
print(f'Número de registros no dataframe de Português: {len(df_por)}')
print(f"Número de registros no dataframe unificado: {len(df)}")

### 2.2 Preparação para unificação final

Pudemos observar, através das etapas anteriores, que tanto no dataset de português como no de matemática, temos os fatores:


Tipos de Dados Consistentes: Todas as colunas correspondentes possuem o mesmo tipo de dado (Dtype) em ambos os arquivos. Por exemplo, a coluna age é do tipo int64 (número inteiro) e a coluna school é do tipo object (texto) nos dois conjuntos. Isso previne erros de cálculo em etapas futuras.

Estrutura Idêntica: Ambos os dataframes possuem exatamente as mesmas 33 colunas, com os mesmos nomes e na mesma ordem.

Ou seja: datasets perfeitamente compatíveis. Poderemos então, pular uma etapa de limpeza detalhada **apenas  para a união** desses datasets, o que só é possível aqui devido a compatibilidade dos datasets nesse conjunto de dados. Em um outro projeto de análise de dados, seria comum encontrar inconsistências (tipos de dados diferentes, valores faltantes, etc.) que exigiriam um tratamento cuidadoso antes de qualquer tentativa de união ou agregação.

In [ ]:
# Concatenar os dois dataframes
df_completo = pd.concat([df_mat, df_por], ignore_index=True)

# Definir colunas-chave e regras de agregação
colunas_identificadoras_aluno = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]

regras_de_agregacao = {
    # Para colunas numéricas de desempenho e hábitos, calculamos a média entre as duas matérias
    'G1': 'mean', 'G2': 'mean', 'G3': 'mean',
    'absences': 'mean', 'failures': 'mean',
    'studytime': 'mean', 'traveltime': 'mean',
    'goout': 'mean', 'health': 'mean', 'famrel': 'mean',
    'Dalc': 'mean', 'Walc': 'mean', 'freetime': 'mean',

    # Para colunas com informações do aluno (que não mudam), pegamos o primeiro registro
    # Não estamos adicionando colunas identificadoras dos alunos pois são iguais
    'guardian': 'first',
    'schoolsup': 'first',
    'famsup': 'first',
    'paid': 'first',
    'activities': 'first',
    'higher': 'first',
    'romantic': 'first'
}

# Agrupar e agregar em um único passo
df_final = df_completo.groupby(colunas_identificadoras_aluno).agg(regras_de_agregacao).reset_index()

print(f"O número final de alunos únicos é: {len(df_final)}")
print("Primeiros 5 registros do dataframe final: ")
df_final.head()

In [ ]:
#Info do dataframe final
df_final.info()

### 2.3. Criando Novas Variáveis (Engenharia de Atributos)

Para ir além das variáveis originais e responder nossas perguntas de forma mais eficaz, decidimos criar novas colunas a partir dos dados existentes. Este processo, conhecido como engenharia de atributos, nos permite construir métricas mais poderosas e alinhadas aos nossos objetivos.

- alcool_total: As colunas Dalc (consumo na semana) e Walc (consumo no fim de semana) são úteis, mas para ter uma visão geral do consumo de um aluno, criamos uma única métrica. A coluna alcool_total combina as duas, mas com um peso diferente: consideramos que a semana tem 5 dias e o fim de semana 2. Essa ponderação nos dá uma estimativa mais realista do consumo semanal total, simplificando as comparações.

- passed: A nota final (G3) varia de 0 a 20. Para análises de alto nível, é mais fácil entender o resultado de forma binária: o aluno passou ou não? Por isso, criamos a coluna passed. Ela transforma a nota em uma categoria ('Sim' ou 'Não'), o que é perfeito para criar visualizações de proporção e para comparar o desempenho de forma clara entre diferentes grupos.

In [ ]:
# Criando a coluna de consumo total de álcool
# A ponderação de 5 para dias de semana e 2 para fim de semana
df_final['alcool_total'] = (5 * df_final['Dalc']) + (2 * df_final['Walc'])

# Criando a coluna de aprovação (nota >= 10)
df_final['passed'] = np.where(df_final['G3'] >= 10, 'Sim', 'Não')

print("Novas colunas criadas com sucesso")
display(df_final[['Dalc', 'Walc', 'alcool_total', 'G3', 'passed']].head())

### 2.4 Pré-processamento de dados (Limpeza)

Vamo começar a investigar a saúde dos nossos dados. Isso envolve procurar por possíveis ruídos (dados mal formatados ou "sujos") e outliers (valores extremos que podem distorcer as análises).

Podemos achar que, se a função .isnull() não retorna nada, os dados estão limpos. No entanto, dados faltantes podem se esconder na forma de textos como '?', 'N/A' ou simplesmente espaços em branco. Para investigar isso, fizemos uma auditoria manual nas principais colunas categóricas usando a função .value_counts().



In [ ]:
colunas_categoricas_auditoria = ['Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'romantic']

print("Analisando valores nas colunas categóricas (identificando se há ruídos)")

# Este loop vai passar por cada coluna da lista e mostrar os valores que ela contém
for coluna in colunas_categoricas_auditoria:
    print(f"\nValores únicos na coluna '{coluna}':")
    print(df_final[coluna].value_counts())
    print("-" * 30)

Felizmente, nosso dataset se mostrou muito limpo e bem estruturado. A auditoria confirmou que não havia ruídos ou valores não padronizados nas colunas, o que demonstra a alta qualidade da base de dados. Mesmo assim, realizar essa verificação foi um passo essencial para garantir a confiabilidade das nossas análises futuras.


Agora, verificando os Outliers são valores que fogem muito do padrão do resto dos dados. Eles são importantes porque podem distorcer médias e outras medidas estatísticas. Usamos boxplots para visualizá-los.

In [ ]:
colunas_para_boxplot = ['absences', 'G3', 'freetime', 'health']

print("INVESTIGAÇÃO DE OUTLIER \n")

# Criamos um loop para gerar um boxplot para cada coluna da lista
for coluna in colunas_para_boxplot:
    plt.figure(figsize=(8, 4)) # Cria uma nova figura para cada gráfico
    sns.boxplot(x=df_final[coluna])
    plt.title(f'Boxplot da coluna: {coluna}', fontsize=14)
    plt.show()

Conclusão da Análise de Outliers: A investigação com boxplots revelou a presença de outliers significativos, principalmente na coluna absences (faltas). Enquanto a maioria dos alunos tem poucas faltas, alguns poucos possuem um número extremamente alto.

Mas, seguindo as diretrizes do projeto, nossa decisão foi não remover esses outliers, pois eles representam um comportamento real e importante de um grupo de alunos. Ignorá-los seria ignorar uma parte da realidade escolar. Em vez disso, vamos gerenciá-los na próxima etapa através da discretização, agrupando-os em uma categoria específica, o que nos permite analisar seu impacto sem que eles distorçam as estatísticas gerais.

### 2.4 Discretização para melhorar análise futura

Para simplificar a análise de variáveis numéricas contínuas e facilitar a visualização de tendências, aplicamos a técnica de discretização. Isso significa transformar números em faixas ou categorias nomeadas.


- Para age: Agrupamos as idades em três faixas (15-16, 17-18, 19+), o que nos permite comparar o comportamento de adolescentes mais novos com os mais velhos de forma mais clara.

- Para absences: Criamos as categorias Nenhuma Falta, Poucas Faltas, Faltas Moderadas e Muitas Faltas. Essa foi nossa estratégia para lidar com os outliers: eles agora pertencem ao grupo "Muitas Faltas", nos permitindo analisar seu comportamento sem distorcer as médias.

- Para G3: Transformamos a nota final em quatro níveis de desempenho_final (Insuficiente, Regular, Bom, Excelente). Isso torna muito mais intuitivo cruzar o desempenho com outros fatores, como o consumo de álcool ou o tempo de estudo.

In [ ]:
# Discretização de 'age' em faixas etárias
faixas_idade = [14, 16, 18, 23]
nomes_faixas_idade = ['15-16 anos', '17-18 anos', '19+ anos']
df_final['faixa_etaria'] = pd.cut(x=df_final['age'], bins=faixas_idade, labels=nomes_faixas_idade, include_lowest=True)

# Discretização de 'absences'(faltas)
faixas_faltas = [-1, 0, 4, 9, 100]
nomes_faixas_faltas = ['Nenhuma Falta', 'Poucas Faltas (1-4)', 'Faltas Moderadas (5-9)', 'Muitas Faltas (10+)']
df_final['faixa_faltas'] = pd.cut(x=df_final['absences'], bins=faixas_faltas, labels=nomes_faixas_faltas)

# Discretização de 'G3' (Nota Final) definindo faixas de desempenho.
faixas_notas = [-1, 9, 13, 17, 20]
nomes_faixas_notas = ['Insuficiente (0-9)', 'Regular (10-13)', 'Bom (14-17)', 'Excelente (18-20)']
df_final['desempenho_final'] = pd.cut(x=df_final['G3'], bins=faixas_notas, labels=nomes_faixas_notas)

print("Exemplos das novas colunas")
display(df_final[['age', 'faixa_etaria', 'absences', 'faixa_faltas', 'G3', 'desempenho_final']].sample(10))

### 2.5 Binarização de variaveis booleanas

Várias colunas do nosso dataset continham respostas de "sim" ou "não". Para podermos usar essa informação em análises numéricas (como em mapas de correlação), transformamos essas respostas em valores binários: 1 para 'yes' e 0 para 'no'.

Essa transformação é poderosa porque nos permite, por exemplo, calcular a média de uma coluna como internet para descobrir o percentual exato de alunos com acesso à internet, ou incluir essas variáveis em análises estatísticas para descobrir novas relações.

In [ ]:
colunas_binarias = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

print("TRANSFORMANDO COLUNAS BINÁRIAS \n")

# Loop para aplicar a transformação a cada coluna
for coluna in colunas_binarias:
    df_final[coluna] = df_final[coluna].map({'yes': 1, 'no': 0})
    print(f"Percentual de 'sim' na coluna '{coluna}': {df_final[coluna].mean():.2%}")

print("\nDados transformados com sucesso!")
display(df_final[colunas_binarias].head())

## 3. Análise Exploratória dos Dados (AED)

Com os dados devidamente limpos, tratados e enriquecidos com novas variáveis, podemos começar. Nesta seção, vamos explorar as principais variáveis, primeiro de forma isolada e depois cruzando informações para encontrar padrões, testar nossas hipóteses e responder às perguntas do nosso projeto.

### 3.1. Análise Univariada

Pra começar a entender o perfil dos alunos neste dataset, a gente decidiu olhar para as variáveis mais importantes de forma individual. Isso nos dá uma base sobre a distribuição de cada característica antes de começar a fazer cruzamentos.

In [ ]:
# Distribuição de frequência das nossas principais variáveis categóricas
print("\n--- Distribuição de Frequências das Variáveis Categóricas ---")

# Lista de colunas para analisar
colunas_categoricas = ['faixa_etaria', 'faixa_faltas', 'desempenho_final', 'passed', 'studytime', 'Mjob', 'Fjob', 'freetime', 'goout', 'famrel', 'health']

for coluna in colunas_categoricas:
    plt.figure(figsize=(10, 5))

    sns.countplot(
        y=coluna,
        data=df_final,
        hue=coluna,
        palette='viridis',
        legend=False,
        order=df_final[coluna].value_counts().index
    )

    plt.title(f'Distribuição de Frequência para: {coluna}', fontsize=15)
    plt.xlabel('Contagem de Alunos')
    plt.ylabel('')
    plt.show()


- Análise dos Gráficos de Frequência:
Ao analisar os gráficos, algumas características do nosso grupo de estudantes ficaram bem claras:

Perfil Etário e Acadêmico: A grande maioria dos alunos se concentra na faixa dos 17-18 anos. No quesito desempenho, vimos que as categorias mais populosas são a "Regular"

Hábitos Sociais: A frequência de saídas (goout) é bem distribuída, mas com uma leve concentração nos níveis 3 e 4 (de 1 a 5), indicando uma vida social ativa para a maioria.

Assiduidade: Um ponto que chamou nossa atenção foi a quantidade de alunos na faixa "Nenhuma Falta", e "Pouca falta" que é o grupo mais numeroso.

In [ ]:
colunas_restantes = [
    'school', 'address', 'famsize', 'Pstatus', 'reason', 'guardian',
    'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
    'internet', 'romantic', 'sex'
]
print("ANÁLISE DESCRITIVA DAS VARIÁVEIS CATEGÓRICAS DE APOIO")

# Loop para analisar cada uma das colunas restantes
for coluna in colunas_restantes:
    print(f"\n----- Análise da Coluna: '{coluna}' -----")

    contagem = df_final[coluna].value_counts()
    percentual = df_final[coluna].value_counts(normalize=True) * 100

    resumo_coluna = pd.DataFrame({
        'Contagem': contagem,
        'Percentual (%)': percentual.round(2)
    })

    display(resumo_coluna)



Além dos gráficos, a gente fez uma análise rápida das outras variáveis de apoio para entender o contexto geral dos alunos. Essa checagem rápida nos mostrou que:

A maioria dos estudantes é da escola Gabriel Pereira (GP), mora em área urbana, tem acesso à internet em casa e afirmou não estar em um relacionamento amoroso.

Um dado muito positivo é que a grande maioria (higher) aspira continuar os estudos e cursar o ensino superior.

Em relação ao suporte, a maioria recebe apoio familiar (famsup), mas não possui aulas de reforço pagas (paid).

Essas informações montam o pano de fundo demográfico e social da nossa amostra e serão úteis para contextualizar as análises seguintes.

### 3.2. Análise Bivariada

Agora, vamos começar a cruzar variáveis para investigar possíveis relações e testar algumas de nossas hipóteses.

#### Pergunta 1: O consumo de álcool afeta o desempenho acadêmico (nota final)?

In [ ]:
plt.figure(figsize=(12, 7))

sns.barplot(
    x='desempenho_final',
    y='alcool_total',
    data=df_final,
    hue='desempenho_final',
    palette='coolwarm',
    legend=False,
    order=['Insuficiente (0-9)', 'Regular (10-13)', 'Bom (14-17)', 'Excelente (18-20)']
)

plt.title('Consumo Médio de Álcool por Faixa de Desempenho', fontsize=16)
plt.xlabel('Desempenho na Nota Final (G3)')
plt.ylabel('Consumo Médio Total de Álcool')
plt.show()


O gráfico de barras é bem direto e confirma nossa principal hipótese. Existe uma relação negativa clara: quanto pior a faixa de desempenho do aluno, maior o seu consumo médio de álcool. A barra do grupo "Insuficiente" é visivelmente a mais alta, e o consumo vai diminuindo progressivamente conforme o desempenho melhora. Isso é uma forte evidência de que o álcool e o sucesso acadêmico caminham em direções opostas.

#### Pergunta 2: A vida social (sair com amigos) está relacionada ao consumo de álcool?

In [ ]:
plt.figure(figsize=(12, 7))
sns.violinplot(x='goout', y='alcool_total', data=df_final, palette='autumn', inner='box')
plt.title('Consumo de Álcool vs. Frequência de Saídas com Amigos', fontsize=16)
plt.xlabel('Frequência de Saídas (1: Muito Baixa - 5: Muito Alta)')
plt.ylabel('Consumo Total de Álcool')
plt.show()

Análise: A relação aqui é bastante clara e positiva. O violinplot mostra que quanto maior a frequência com que os estudantes saem com os amigos (goout), maior tende a ser o consumo de álcool (alcool_total). A gente pode ver não só a mediana (o ponto branco) subindo, mas a distribuição toda se deslocando para cima. Isso é um forte indicativo de que o contexto social influencia diretamente os hábitos de consumo.

### Pergunta 3: Sair mais leva a faltar mais?

In [ ]:
# Gráfico 2: Relação entre Frequência de Saídas e Número de Faltas
plt.figure(figsize=(12, 7))

sns.boxplot(x='goout', y='absences', data=df_final, palette='coolwarm')
plt.title('Número de Faltas vs. Frequência de Saídas com Amigos', fontsize=16)
plt.xlabel('Frequência de Saídas (1: Muito Baixa - 5: Muito Alta)')
plt.ylabel('Número de Faltas')
plt.show()

Análise: Para testar essa conexão, usamos um boxplot. Ele nos ajuda a ver a distribuição das faltas para cada nível de "saídas". Podemos notar que a linha da mediana (o traço no meio da caixa) sobe um pouco à medida que a frequência de saídas aumenta. Isso indica que, em geral, quem sai mais tende a ter, sim, mais faltas. Também é interessante ver que os outliers (os pontos sozinhos) aparecem em todos os grupos, mostrando que alunos com um número extremo de faltas existem em qualquer perfil social, e alunos que disseram sair mais (5.0) também não são o grupo com maior mediana de consumo de alcool

### Pergunta 4: A assiduidade (presença nas aulas) tem impacto no desempenho do aluno?


In [ ]:
tabela_faltas_desempenho = pd.crosstab(df_final['faixa_faltas'], df_final['desempenho_final'], normalize='index') * 100

tabela_faltas_desempenho.plot(
    kind='barh',
    stacked=True,
    figsize=(14, 8),
    colormap='RdYlGn'
)

plt.title('Composição do Desempenho Acadêmico por Faixa de Faltas', fontsize=16)
plt.xlabel('Percentual de Alunos (%)')
plt.ylabel('Faixa de Faltas')
plt.legend(title='Desempenho Final')
plt.show()

Analisando o gráfico, a gente consegue ver uma história clara sobre como a assiduidade está ligada ao nível de desempenho dos alunos. Cada barra horizontal representa 100% dos alunos dentro de uma faixa de faltas, e as cores nos mostram como o desempenho se distribui dentro desse grupo.

O que mais chama a atenção é a mudança na proporção das cores, principalmente as verdes (que representam as melhores notas).

No grupo "Nenhuma Falta", as barras verde-claro ("Bom") e verde-escuro ("Excelente") juntas somam uma porção significativa, quase 30% da barra. Isso indica que uma parte considerável dos alunos mais assíduos atinge os níveis mais altos de desempenho.

No grupo "Muitas Faltas (10+)", o cenário muda drasticamente. As barras verdes se tornam muito pequenas, quase desaparecendo. Em contrapartida, a barra laranja ("Regular") se expande, ocupando a maior parte do espaço. A barra vermelha ("Insuficiente") também tem uma presença forte e ligeiramente maior que no grupo sem faltas.

Conclusão da Análise: A nossa interpretação é que o principal impacto do aumento de faltas não é necessariamente "reprovar mais", mas sim uma queda na excelência acadêmica. Alunos que faltam muito perdem a chance de alcançar um desempenho "Bom" ou "Excelente", ficando, na melhor das hipóteses, com um desempenho "Regular". A assiduidade, portanto, se mostra um fator crucial para se alcançar os melhores resultados.

### 3.3. Análise Multivariada

Para ter uma visão panorâmica de todas as relações numéricas de uma só vez, a gente gerou um mapa de calor. Esta ferramenta visual é poderosa: as cores mais claras (amarelo) indicam uma correlação positiva forte (quando uma variável sobe, a outra também sobe), enquanto as cores mais escuras (roxo) indicam uma correlação fraca ou negativa.

In [ ]:
# Selecionando apenas colunas numéricas para a correlação
numeric_df_final = df_final.select_dtypes(include=np.number)

correlation_matrix = numeric_df_final.corr()

# Plotando o heatmap
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=False, cmap='viridis')
plt.title('Mapa de Calor de Correlação (Incluindo Variáveis Binárias)', fontsize=18)
plt.show()

print('--- Principais Correlações com a Nota Final (G3) ---')
display(correlation_matrix['G3'].sort_values(ascending=False).to_frame())

Os melhores indicadores do desempenho final de um aluno são suas notas anteriores (G1 e G2). A correlação altíssima (acima de 0.83) mostra que o histórico acadêmico recente é o fator mais decisivo para o sucesso.

Logo em seguida, encontramos um grupo de fatores ligados ao ambiente e à ambição do aluno:

Desejo de ir para a faculdade (higher): Com uma correlação de 0.27, a aspiração de continuar os estudos se mostra um importante fator motivacional associado a notas melhores.

Educação dos Pais (Medu e Fedu): Com correlações de 0.23 e 0.18, respectivamente, o nível educacional dos pais confirma sua influência positiva, provavelmente refletindo um ambiente familiar que valoriza mais os estudos.

Tempo de Estudo (studytime): Como esperado, mais tempo de estudo também tem uma correlação positiva (0.19) com a nota final.

- Os Fatores de Maior Impacto Negativo (O que atrapalha o aluno)

Do lado negativo, o fator que mais pesa contra o aluno é, de longe, o número de reprovações anteriores (failures), com uma forte correlação negativa de -0.40. Isso sugere que dificuldades passadas são um grande obstáculo para o sucesso futuro.

Depois, confirmamos numericamente a história que nossa análise vinha contando sobre o estilo de vida:

Consumo de Álcool (alcool_total, Walc, Dalc): Todas as métricas de álcool apresentam uma correlação negativa consistente (em torno de -0.14 a -0.16), validando a hipótese de que o consumo de álcool está associado a notas piores.

Vida Social (goout): A frequência de saídas também tem um impacto negativo (-0.10), reforçando a ideia de que o excesso de vida social pode competir com as responsabilidades acadêmicas.

- A DescobertA das Faltas (absences)
Um ponto que chamou muito a nossa atenção foi a correlação quase nula da coluna absences (-0.06). Isso parece contradizer nossa teoria anterior, que deduzia um impacto das faltas no desempenho.

O que isso nos diz? Que a relação entre faltas e notas não é uma linha reta simples (ou seja, não é uma correlação linear forte). O impacto negativo das faltas não acontece de forma gradual; ele se torna realmente drástico apenas nos extremos (no grupo de "Muitas Faltas"). Esse é um padrão complexo que a nossa análise com o gráfico de barras conseguiu capturar muito melhor do que um simples número de correlação. Isso mostra a importância de usar visualizações para entender as nuances dos dados, em vez de confiar apenas em uma única métrica estatística.

### Visualização Focada
O heatmap completo é ótimo para uma visão geral. Como o foco principal é o desempenho (G3), decidimos adicionar uma visualização complementar mostrando apenas as correlações com essa variável, de forma ordenada. Isso destaca imediatamente os fatores mais importantes.

In [ ]:
# Mostra as correlações com G3 em um gráfico de barras
plt.figure(figsize=(10, 12))
correlation_matrix['G3'].sort_values(ascending=False).drop('G3').plot(kind='barh')
plt.title('Correlação das Variáveis com a Nota Final (G3)')
plt.show()

In [ ]:
# Importar as bibliotecas necessárias
import seaborn as sns
import matplotlib.pyplot as plt

# Criar o gráfico facetado usando catplot
g = sns.catplot(
    data=df_final,
    x='desempenho_final',   # O que queremos comparar no eixo X
    y='goout',              # O que queremos medir no eixo Y
    col='faixa_faltas',     # A variável que vai criar as colunas (facetas)
    kind='bar',             # Tipo de gráfico
    palette='viridis',        # Paleta de cores
    height=5,               # Altura de cada subplot
    aspect=0.8,             # Proporção entre largura e altura
    order=['Insuficiente (0-9)', 'Regular (10-13)', 'Bom (14-17)', 'Excelente (18-20)'] # Ordem das barras no eixo X
)

# Ajustes e títulos para melhor visualização
g.fig.suptitle('O Efeito em Cadeia: Vida Social, Faltas e Desempenho Final', y=1.03, fontsize=16)
g.set_axis_labels('Desempenho Final', 'Frequência Média de Saídas')
g.set_titles("Faltas: {col_name}")
g.set_xticklabels(rotation=45, ha='right')
plt.tight_layout(rect=[0, 0, 1, 0.97])

# Mostrar o gráfico
plt.show()

## 4. Conclusões da Análise

Após um processo iterativo de preparação, engenharia de features, e análise exploratória, chegamos a uma compreensão profunda dos fatores que cercam o consumo de álcool e o desempenho acadêmico dos estudantes na amostra. A decisão de unificar os datasets de forma inteligente e a criação de novas variáveis, como as faixas de desempenho (desempenho_final) e de faltas (faixa_faltas), foram cruciais para revelar padrões complexos que não eram óbvios nos dados brutos.

### 4.1. Respondendo às Perguntas Principais

Com base nas visualizações e análises, podemos agora responder diretamente às nossas três perguntas de pesquisa  centrais.

1. Qual a relação entre o consumo de álcool e o desempenho acadêmico?

Conclusão: Existe uma relação negativa clara e consistente. Nossa análise confirmou a hipótese de que um maior consumo de álcool está associado a um pior desempenho acadêmico.

Evidência: O gráfico de barras comparando o consumo_total_alcool pela nossa variável desempenho_final mostrou que a média de consumo do grupo "Insuficiente" foi significativamente maior do que a dos grupos "Bom" e "Excelente".

2. Como o estilo de vida social e os hábitos (sair, faltar) se conectam com o desempenho?

Conclusão: Identificamos uma "cadeia de comportamentos de risco" com forte impacto negativo. Um estilo de vida mais ativo socialmente (goout) está ligado a maiores níveis de consumo de álcool e a um maior número de faltas, e este último, por sua vez, está diretamente associado a um pior desempenho final.

Evidência: Nossa análise em cadeia conectou os pontos: os gráficos mostraram que mais saídas levam a mais álcool e mais faltas; e a análise final com o gráfico de barras empilhadas provou que mais faltas levam a uma proporção drasticamente maior de alunos com desempenho "Insuficiente".

3. Quais outros fatores demonstram forte influência sobre o sucesso acadêmico?

Conclusão: Além do álcool e das faltas, dois grandes grupos de fatores se destacaram: o histórico acadêmico do aluno e o ambiente familiar e de suporte.

Evidência (Histórico): O mapa de correlação e a lista ordenada mostraram que as notas anteriores (G1, G2) e, principalmente, o número de reprovações passadas (failures) são os preditores mais fortes da nota final.

Evidência (Ambiente): A análise mostrou que um nível educacional mais alto dos pais (Medu, Fedu) e o desejo do próprio aluno de cursar o ensino superior (higher) também demonstraram uma correlação positiva importante com o sucesso.



### 4.2. Resumo das Descobertas e Conclusões Finais

A "Cadeia de Risco" é o principal insight: A descoberta mais relevante não foi um fator isolado, mas a conexão entre eles. A vida social ativa (goout), quando não equilibrada, parece ser o gatilho para um aumento no consumo de álcool e na ausência escolar, formando um caminho que leva a dificuldades acadêmicas.

O Paradoxo das Faltas: Uma descoberta inesperada foi que a simples taxa de "aprovação" parecia aumentar com as faltas, um resultado que contraria o senso comum. No entanto, ao aprofundar a análise com nossa variável desempenho_final, resolvemos o paradoxo: um alto número de faltas praticamente elimina a chance de um aluno alcançar um desempenho "Bom" ou "Excelente", mesmo que alguns consigam uma aprovação com nota mínima.

Fatores de Proteção: Em contrapartida, um ambiente familiar com maior capital educacional e a própria ambição do aluno em continuar os estudos (higher) funcionam como importantes fatores de proteção, associados a melhores resultados.